In [3219]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import math
import copy
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff


from enum import Enum
from scipy.stats import truncnorm
from scipy.stats import norm

import seaborn as sns


# Auction Simulation


## Enums and Static Values


In [3220]:
class BIDDER_TYPE(Enum):
    influencer = 1
    reactor = 2


In [3221]:
new_line = '\n'
new_line_space = '\n' + '   '


## Helper Functions


In [3222]:
# returns n values, normally distributed:
# mean: average value
# std: standard deviation
# on range [low, upp] (squashed into that range)
def get_truncated_normal(mean, std, count, low=0, upp=1):
    a, b = (low - mean) / std, (upp - mean) / std

    return truncnorm.rvs(
        a, b, loc=mean, scale=std, size=count)


In [3223]:
# returns the n-th percentile of a normal distribution with:
# mean: average value
# n: n-th percentile

# Ex: 95th percentile -> point which 95% of the numbers are below
def get_nth_percentile(std: float, mean: float, n: int):
    return norm.ppf(n / 100.0, loc=mean, scale=std) # percent-point-function 


In [3224]:
# infers bidder type based on his std_self value
def get_bidder_type(std_self, std):
    std_self_trunc = std_self / std # std truncated to range [0,1]

    # low std -> much confindence in own start value estimate
    if std_self_trunc > 0.5:
        return BIDDER_TYPE.reactor

    return BIDDER_TYPE.influencer


In [3225]:
# returns average of values in list
def average_value(values: list[int] or list[float]):
    return sum(values) / len(values)


In [3226]:
# returns:
# std_self - how little he trusts his original value estimate
# std_others - how little he trusts other people's bids as estimates
# ..the values are negatively correlated
def calculate_stds(private_info, consensus_bias, desire_coef, risk_coef, std_private_values):
    std_self_coef = average_value(
        [1-private_info, 1-consensus_bias, desire_coef, risk_coef])
    std_others_coef = 1 - std_self_coef

    std_self = std_self_coef * std_private_values
    std_others = std_others_coef * std_private_values

    return std_self, std_others


In [3227]:
# returns distribution of bidder's belief of other people's values as list of random floats
def get_value_belief_dist(private_value, std, no_bidders):
    # loc: mean
    # scale: std (sigma)
    # size: how many numbers to generate
    return np.random.normal(loc=private_value, scale=std, size=no_bidders)


In [3228]:
# how much the bidder trusts incoming information at time t
def get_std_incoming(std_others: float, t: int):
    time_coef = 0.000000001 + t * 0.05 # VILJUM FINNA ÞETTA FALL!
    return std_others * time_coef

In [3229]:
def update_belief_set2(bidder, no_bidders, time, all_bids):
    std_incoming =  bidder.std_others # TODO: use function instead
    std_prior = bidder.std
    n = time

    std_post = math.sqrt(1 / ( (n / math.pow(std_incoming, 2)) + (1 / math.pow(std_prior, 2)) ))


    mu_prior = bidder.curr_value
    x_mean = average_value(list(map(lambda bid: bid.amount, all_bids)))

    mean_post = math.pow(std_post, 2) * ( (mu_prior / math.pow(std_prior, 2)) + (x_mean / math.pow(std_incoming, 2)) )

    bidder.curr_value = math.floor(mean_post)
    bidder.std = std_post

    # get updated value distribution using new mean and std
    bidder.value_belief_distribution = get_value_belief_dist(
        bidder.curr_value, bidder.std, no_bidders)


### Plotting


In [3230]:
# plots all bidder's belief set distributions in one graph
def plot_belief_distributions(belief_sets, title):

    plt.figure(figsize=(20, 12))

    hist_data = list(belief_sets)
    group_labels = list(map(lambda x: str(x), range(0, len(belief_sets))))

    fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False, curve_type="kde", bin_size=50)
    fig.update_layout(title=title)
    fig.show()


## Classes


In [3231]:
class Auction:
    def __init__(self, id, N, reserve, start_bid):
        self.id = id

        # static values
        self.N = N
        self.reserve = reserve
        self.start_bid = start_bid
        self.bidders = None

        # dynamic values
        self.t = 0
        self.curr_bid = start_bid
        self.all_bids = []

    def __str__(self) -> str:

        attribute_strings = (
            'id: ' + self.id + new_line_space +
            'no. bidders: ' + str(self.N) + new_line_space +
            'reserve: ' + str(self.reserve) + new_line
        )

        return (
            'Auction(' + new_line_space +
            attribute_strings +
            ')' + new_line
        )


In [3232]:

class Bidder:
    def __init__(self, name, bidder_type, predef_value, std_self, std_others, desire_coef, value_belief_distribution):
        self.name = name
        self.bidder_type = bidder_type  # influencer bidder or reactor
        self.predef_value = predef_value  # bidder's estimated value of item pre-auction

        # how much does the bidder want the item at the start?
        self.desire_coef = desire_coef
        self.curr_value = predef_value  # bidders updated in-auction value
        self.is_active = True  # all bidders start active
        self.no_bids_submitted = 0  # no bids submitted by bidder
        self.max_raise = get_nth_percentile(std_self, predef_value, 95) # the maximum amount he will update his value to (95th percentile)


        # what he thinks other bidder's values are
        self.value_belief_distribution = value_belief_distribution

        # --- std coefficients ---

        # static
        self.std_self = std_self  # how much bidder trusts his original value estimate
        self.std_others = std_others  # how much the bidder trusts incoming information
        
        # dynamic
        self.std = std_self  # how much bidder trusts his current value estimate

    def __str__(self) -> str:

        attribute_strings = (
            'name: ' + self.name + new_line_space +
            'predef_value: ' + str(self.predef_value) + new_line_space +
            'curr_value: ' + str(self.curr_value) + new_line_space +
            'std_self: ' + str(self.std_self) + new_line_space +
            'std_others: ' + str(self.std_others) + new_line_space +
            'std: ' + str(self.std) + new_line_space +
            'bidder_type: ' + str(self.bidder_type) + new_line_space +
            'is_active: ' + str(self.is_active) + new_line_space +
            'max_raise: ' + str(self.max_raise) + new_line
        )

        return (
            'Bidder(' + new_line_space +
            attribute_strings +
            ')' + new_line
        )


In [3233]:
class Bid:
    def __init__(self, amount: int, bidder: Bidder):
        self.amount = amount
        self.bidder = bidder

    def __str__(self) -> str:
        return 'Bid(amount=' + str(self.amount) + ' ,bidder=' + str(self.bidder) + ')'


## Simulation Functions


In [3234]:
def get_bidder_bid(curr_bid: Bid, curr_time: int, bidder: Bidder, no_bidders: int):
    bid_amount = 0

    # bidder has reached his maximum coming in to the auction
    # OR
    # current bid is higher than his current estimated value
    if((curr_bid.amount > bidder.max_raise) | (curr_bid.amount > bidder.curr_value)):
        # bidder opts out of the auction
        bidder.is_active = False
        return 0


    # is bidder ready to place a bid?
    is_time_to_bid = (
        (bidder.bidder_type == BIDDER_TYPE.influencer)
        |
        (
            # bids if time 90% of T or bid 90% of value
            (bidder.bidder_type == BIDDER_TYPE.reactor)
            &
            ((curr_time > (no_bidders / 2)) | # half of bidders have on avg. bid
             (curr_bid.amount > bidder.curr_value * 0.9))
        )
    )
    

    
    if (is_time_to_bid & (curr_bid.bidder != bidder) & (curr_bid.amount < bidder.curr_value)):
        # bid random on range [current bid, halfway from current bid to own value]
        bid_amount = rd.randint(curr_bid.amount, math.floor(
            curr_bid.amount + (bidder.curr_value - curr_bid.amount) / 2))

    return bid_amount


In [3235]:

def run_auction(auction):
    auction.curr_bid = auction.start_bid
    auction.t = 0
    no_more_bids = False

    while (not no_more_bids):
        bids = []

        for bidder in auction.bidders:
            if(bidder.is_active):
                bid_amount = get_bidder_bid(
                    auction.curr_bid, auction.t, bidder, auction.N)

                if (bid_amount > auction.curr_bid.amount):
                    bids.append(Bid(bid_amount, bidder))
        
        if (len(bids) > 0):
            # get maximum of the placed bids at time t and set as current bid
            max_bid = max(bids, key=lambda bid: bid.amount)
            auction.curr_bid = max_bid
            auction.all_bids.append(max_bid)

            # update bidder no. bids
            auction.curr_bid.bidder.no_bids_submitted += 1

            # update each active bidder's belief set
            for bidder in auction.bidders:
                if(bidder.is_active):
                    update_belief_set2(
                        bidder=bidder, no_bidders=auction.N, time=auction.t, all_bids=auction.all_bids)

        else:
            no_more_bids = True

        auction.t += 1

    return auction.curr_bid


In [3236]:
def run_simulation(no_iterations):
    # mean and standard deviation of private values
    # std: how affiliated are the private values?? !!TEST!!
    avg = 1000
    std = avg * 0.2
    winning_bids = []
    all_original_bidders = []
    all_final_bidders = []
    for i in range(0, no_iterations):
        auction = Auction(id='b'+str(i+1), N=15,
                          reserve=avg * 0.8, start_bid=Bid(0, None))

        bidder_private_values = [math.floor(x) for x in get_truncated_normal(
            mean=avg, std=std, count=auction.N, low=0, upp=1000000)]
        bidder_private_infos = get_truncated_normal(
            mean=0.5, std=0.25, count=auction.N)
        bidder_consensus_bias = get_truncated_normal(
            mean=0.5, std=0.25, count=auction.N)
        bidder_desires = get_truncated_normal(
            mean=0.5, std=0.25, count=auction.N)
        bidder_risk_coefs = get_truncated_normal(
            mean=0.5, std=0.25, count=auction.N)

        bidders = []
        for i in range(0, auction.N):
            std_self, std_others = calculate_stds(
                bidder_private_infos[i], bidder_consensus_bias[i], bidder_desires[i], bidder_risk_coefs[i], std)
            bidder_type = get_bidder_type(std_self, std)

            bidders.append(Bidder(
                name='b'+str(i+1),
                bidder_type=bidder_type,
                predef_value=bidder_private_values[i],
                std_self=std_self,
                std_others=std_others,
                desire_coef=bidder_desires[i],
                value_belief_distribution=get_value_belief_dist(bidder_private_values[i], std_self, auction.N)))

        original_bidders = copy.deepcopy(bidders)
        all_original_bidders.append(original_bidders)

        auction.bidders = bidders

        winning_bids.append(run_auction(auction))
        all_final_bidders.append(auction.bidders)

    return winning_bids, all_final_bidders, all_original_bidders


## Simulation


In [3237]:
winning_bids, all_final_bidders, all_original_bidders = run_simulation(500)


2196.159958810183
1337.149853689042
1638.6652495116455
3017.0047310536484
1567.0059325610598
1148.8637436830934
1308.165819285257
720.8050744738684
2090.062595541162
2212.999581521881
1803.1458900574928
1693.0932838340436
1655.607571541372
886.3431806804246
2002.6372795840998
1146.0882092310746
1172.8706659805453
1010.3446882211707
1103.830529161022
968.7871711610217
994.6636808611701
943.5584718784587
685.1446352358735
1039.0427384851648
1109.2934596650528
1132.9478192148404
1052.906316807608
1106.026441908924
852.3532307687035
1089.826245362114
685.7660694022004
916.6982067033446
641.5929519875938
632.110304331744
619.0076998054778
760.5088659051909
636.4474224623298
564.2476319980262
627.4102524743907
662.2656276553857
713.1853406066269
666.337960893089
714.2465551688803
756.7820018627522
663.2988788627315


In [3238]:

# for i in range(0,len(all_original_bidders)):
#     plot_belief_distributions(list(map(lambda bidder: bidder.value_belief_distribution,
#                                  all_original_bidders[i])), 'Bidder\'s (Original) Belief Distributions')
#     plot_belief_distributions(list(map(lambda bidder: bidder.value_belief_distribution,
#                                        all_final_bidders[i])), 'Bidder\'s (Final) Belief Distributions')


<Figure size 2000x1200 with 0 Axes>

<Figure size 2000x1200 with 0 Axes>

In [3239]:
auction_results = []

for i in range(0, len(winning_bids)):
    winning_bidder = winning_bids[i].bidder
    all_but_winner = filter(lambda bidder: bidder.name !=
                            winning_bidder.name, all_original_bidders[i])
    average_loser_value = average_value(
        list(map(lambda losing_bidder: losing_bidder.curr_value, all_but_winner)))
    winner_curse = average_loser_value - winning_bids[i].amount

    auction_result = {
        'winner_curse': winner_curse,
        'winner_amount': winning_bids[i].amount,
        'winner_utility': winning_bids[i].bidder.curr_value - winning_bids[i].amount,
        'winner_type': winning_bids[i].bidder.bidder_type,
        'winner_no_bids_submitted': winning_bids[i].bidder.no_bids_submitted,
        'winner_std': winning_bids[i].bidder.std,
        'winner_std_self': winning_bids[i].bidder.std_self
    }
    auction_results.append(auction_result)

df = pd.DataFrame(auction_results)


In [3240]:
fig = px.scatter(df, x="winner_amount", y="winner_curse", color="winner_type", trendline='ols',
                 opacity=0.3, title='Winner\'s Curse vs. Amount Paid')
fig.update_traces(marker_size=5)
fig.update_xaxes(title='Amount Paid for Item')
fig.update_yaxes(title='Average Loser Value - Amount Paid')
fig.show()

In [3241]:
fig = px.scatter(df, x="winner_amount", y="winner_utility", color="winner_type", trendline='ols',
                 opacity=0.3, title='Winner Utility vs. Amount Paid')
fig.update_traces(marker_size=5)
fig.update_xaxes(title='Amount Paid for Item')
fig.update_yaxes(title='Winner Utility')
fig.show()

In [3242]:
# ---WINNING BIDDER TYPE---

fig = px.bar(map(lambda type: str(type), df['winner_type']))
fig.update_xaxes(title='Winner Bidder Type')
fig.update_yaxes(title='Count')
fig.show()


In [3243]:
fig = px.scatter(df, x="winner_std", y="winner_std_self", color="winner_type",
                 opacity=0.3, title='Winner Std Self vs. Final Std')
fig.update_traces(marker_size=5)
fig.update_xaxes(title='Winner Final Std')
fig.update_yaxes(title='Winner Std Self')
fig.show()


In [3244]:
# ---WINNING BIDDER NO BIDS SUBMITTED---
fig = px.histogram(map(lambda no: str(no), df['winner_no_bids_submitted']), color=df['winner_type'])

fig.update_xaxes(title='No. Bids Submitted')
fig.update_yaxes(title='Count')
fig.show()
